In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras import Model
import pandas as pd
import numpy as np
from sklearn import preprocessing
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
output_notebook()

Using TensorFlow backend.


Loading BokehJS ...

#### Load and prepare data

In [2]:

frame_file = 'paths.csv'
frame_text = 'path'
frame = pd.read_csv(frame_file)
frame = frame[frame.path != 'NO_PATH_A']
#frame = frame[frame.most_frequent_percentage >= 0.6]


data_frame = pd.read_csv('data.csv')
data_frame = data_frame[data_frame.sentence.isin(frame.sentence.values.tolist())]
def build_source(label, df):
    df = df[df['class'] == label]
    return ColumnDataSource(data=dict(
    x=df.x.values.tolist(),
    y=df.y.values.tolist(),
    sentence=df.sentence.tolist(),
    path=df.path.tolist(),
    label=df['class'].tolist()))


In [3]:
vocab_size=5000
maxlen = 10

# transform labels into 3d tensors for lstm
texts = frame[frame_text].values.tolist()
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts)
input_train = tokenizer.texts_to_sequences(texts)
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_train = input_train.reshape((len(texts),maxlen,1))

# create target labels, one for each path
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(frame.path.tolist())
y_train =  to_categorical(label_encoder.transform(frame.path.tolist()))

y_train[0].shape

(2903,)

#### Train the network

In [4]:
def get_desc():
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.figure(figsize=(15,10))
    plt.plot(history.history['acc'],color='green')
    plt.plot(history.history['val_acc'],linestyle='--',color='yellowgreen')
    plt.plot(history.history['loss'],color='red')
    plt.plot(history.history['val_loss'],color='orangered',linestyle='--')
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.legend(['acc', 'val_acc', 'loss', 'val_loss'])
    plt.show()
    plt.figure(figsize=(20,10))

In [ ]:
from keras.layers import Dense, LSTM, Dropout, Embedding,Flatten,Bidirectional, GlobalMaxPooling1D
from keras.models import Sequential
from keras import regularizers

model = Sequential()
#model.add(LSTM(512,return_sequences=True,input_shape=(maxlen,1)))
model.add(LSTM(64,return_sequences=True,input_shape=(maxlen,1)))
model.add(LSTM(64,return_sequences=False,input_shape=(maxlen,1)))
#model.add(LSTM(512,return_sequences=False,input_shape=(maxlen,1)))
model.add(Dense(2903,activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(input_train, y_train,
epochs=1000, batch_size=64,shuffle=True, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10, 64)            16896     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_2 (Dense)              (None, 2903)              188695    
Total params: 238,615
Trainable params: 238,615
Non-trainable params: 0
_________________________________________________________________
Train on 2912 samples, validate on 729 samples
Epoch 1/1000
2912/2912 [==============================] - 2s 855us/step - loss: 7.9790 - acc: 0.0361 - val_loss: 7.9664 - val_acc: 0.0412
Epoch 2/1000
2912/2912 [==============================] - 1s 508us/step - loss: 7.7684 - acc: 0.0429 - val_loss: 8.9903 - val_acc: 0.0412
Epoch 3/1000
2912/2912 [==============================] - 2s 537us/step - los

2912/2912 [==============================] - 2s 550us/step - loss: 2.4715 - acc: 0.4670 - val_loss: 11.5303 - val_acc: 0.1619
Epoch 55/1000
2912/2912 [==============================] - 1s 509us/step - loss: 2.4377 - acc: 0.4729 - val_loss: 11.5404 - val_acc: 0.1591
Epoch 56/1000
2912/2912 [==============================] - 2s 533us/step - loss: 2.4226 - acc: 0.4571 - val_loss: 11.6127 - val_acc: 0.1646
Epoch 57/1000
2912/2912 [==============================] - 2s 538us/step - loss: 2.3666 - acc: 0.4736 - val_loss: 11.5927 - val_acc: 0.1687
Epoch 58/1000
2912/2912 [==============================] - 1s 500us/step - loss: 2.3195 - acc: 0.4894 - val_loss: 11.6665 - val_acc: 0.1619
Epoch 59/1000
2912/2912 [==============================] - 2s 559us/step - loss: 2.2868 - acc: 0.4911 - val_loss: 11.7098 - val_acc: 0.1632
Epoch 60/1000
2912/2912 [==============================] - 1s 502us/step - loss: 2.2642 - acc: 0.4911 - val_loss: 11.6693 - val_acc: 0.1742
Epoch 61/1000
2912/2912 [=========

2912/2912 [==============================] - 2s 559us/step - loss: 1.2873 - acc: 0.6271 - val_loss: 12.4565 - val_acc: 0.1934
Epoch 113/1000
2912/2912 [==============================] - 2s 554us/step - loss: 1.2962 - acc: 0.6226 - val_loss: 12.4231 - val_acc: 0.1920
Epoch 114/1000
2912/2912 [==============================] - 2s 541us/step - loss: 1.2718 - acc: 0.6322 - val_loss: 12.4690 - val_acc: 0.1907
Epoch 115/1000
2912/2912 [==============================] - 2s 523us/step - loss: 1.2529 - acc: 0.6367 - val_loss: 12.4727 - val_acc: 0.1962
Epoch 116/1000
2912/2912 [==============================] - 1s 510us/step - loss: 1.2405 - acc: 0.6380 - val_loss: 12.4613 - val_acc: 0.1962
Epoch 117/1000
2912/2912 [==============================] - 2s 566us/step - loss: 1.2494 - acc: 0.6305 - val_loss: 12.4635 - val_acc: 0.1934
Epoch 118/1000
2912/2912 [==============================] - 2s 531us/step - loss: 1.2606 - acc: 0.6223 - val_loss: 12.4547 - val_acc: 0.1879
Epoch 119/1000
2912/2912 [==

2912/2912 [==============================] - 2s 534us/step - loss: 0.9024 - acc: 0.6854 - val_loss: 12.6365 - val_acc: 0.1962
Epoch 171/1000
2912/2912 [==============================] - 2s 551us/step - loss: 0.9213 - acc: 0.6693 - val_loss: 12.6450 - val_acc: 0.1879
Epoch 172/1000
2912/2912 [==============================] - 2s 570us/step - loss: 0.9865 - acc: 0.6463 - val_loss: 12.6352 - val_acc: 0.1975
Epoch 173/1000
2912/2912 [==============================] - 2s 528us/step - loss: 0.9264 - acc: 0.6696 - val_loss: 12.6338 - val_acc: 0.1962
Epoch 174/1000
2912/2912 [==============================] - 2s 517us/step - loss: 0.9122 - acc: 0.6758 - val_loss: 12.6260 - val_acc: 0.2030
Epoch 175/1000
2912/2912 [==============================] - 2s 553us/step - loss: 0.8781 - acc: 0.6841 - val_loss: 12.6187 - val_acc: 0.2003
Epoch 176/1000
2912/2912 [==============================] - 2s 518us/step - loss: 0.8687 - acc: 0.6906 - val_loss: 12.6160 - val_acc: 0.2003
Epoch 177/1000
2912/2912 [==

2912/2912 [==============================] - 2s 540us/step - loss: 0.7280 - acc: 0.7109 - val_loss: 12.6562 - val_acc: 0.2044
Epoch 229/1000
2912/2912 [==============================] - 1s 509us/step - loss: 0.7143 - acc: 0.7157 - val_loss: 12.6567 - val_acc: 0.2030
Epoch 230/1000
2912/2912 [==============================] - 1s 499us/step - loss: 0.7056 - acc: 0.7181 - val_loss: 12.6532 - val_acc: 0.2016
Epoch 231/1000
2912/2912 [==============================] - 1s 502us/step - loss: 0.7124 - acc: 0.7143 - val_loss: 12.6633 - val_acc: 0.2071
Epoch 232/1000
2912/2912 [==============================] - 1s 477us/step - loss: 0.7001 - acc: 0.7163 - val_loss: 12.6540 - val_acc: 0.2099
Epoch 233/1000
2912/2912 [==============================] - 1s 503us/step - loss: 0.7068 - acc: 0.7112 - val_loss: 12.6539 - val_acc: 0.2099
Epoch 234/1000
2912/2912 [==============================] - 2s 561us/step - loss: 0.6990 - acc: 0.7146 - val_loss: 12.6546 - val_acc: 0.2044
Epoch 235/1000
2912/2912 [==

2912/2912 [==============================] - 1s 498us/step - loss: 0.6305 - acc: 0.7273 - val_loss: 12.6665 - val_acc: 0.2030
Epoch 287/1000
2912/2912 [==============================] - 2s 574us/step - loss: 0.6249 - acc: 0.7297 - val_loss: 12.6652 - val_acc: 0.2071
Epoch 288/1000
2912/2912 [==============================] - 2s 522us/step - loss: 0.6223 - acc: 0.7339 - val_loss: 12.6639 - val_acc: 0.2058
Epoch 289/1000
2912/2912 [==============================] - 1s 511us/step - loss: 0.6127 - acc: 0.7325 - val_loss: 12.6681 - val_acc: 0.2030
Epoch 290/1000
2912/2912 [==============================] - 2s 542us/step - loss: 0.6046 - acc: 0.7390 - val_loss: 12.6684 - val_acc: 0.2016
Epoch 291/1000
2912/2912 [==============================] - 2s 529us/step - loss: 0.6105 - acc: 0.7328 - val_loss: 12.6639 - val_acc: 0.2030
Epoch 292/1000
2912/2912 [==============================] - 2s 520us/step - loss: 0.6026 - acc: 0.7345 - val_loss: 12.6636 - val_acc: 0.2044
Epoch 293/1000
2912/2912 [==

2912/2912 [==============================] - 2s 519us/step - loss: 0.6124 - acc: 0.7373 - val_loss: 12.6835 - val_acc: 0.1989
Epoch 345/1000
2912/2912 [==============================] - 2s 568us/step - loss: 0.6204 - acc: 0.7246 - val_loss: 12.6703 - val_acc: 0.2030
Epoch 346/1000
2912/2912 [==============================] - 2s 539us/step - loss: 0.5942 - acc: 0.7328 - val_loss: 12.6605 - val_acc: 0.2044
Epoch 347/1000
2912/2912 [==============================] - 2s 534us/step - loss: 0.6092 - acc: 0.7301 - val_loss: 12.6749 - val_acc: 0.2016
Epoch 348/1000
2912/2912 [==============================] - 2s 552us/step - loss: 0.6066 - acc: 0.7363 - val_loss: 12.6843 - val_acc: 0.1975
Epoch 349/1000
2912/2912 [==============================] - 2s 515us/step - loss: 0.6221 - acc: 0.7188 - val_loss: 12.6705 - val_acc: 0.2030
Epoch 350/1000
2912/2912 [==============================] - 2s 525us/step - loss: 0.5768 - acc: 0.7400 - val_loss: 12.6924 - val_acc: 0.1934
Epoch 351/1000
2912/2912 [==

In [ ]:
get_desc()

#### Prepare the embeddings

In [6]:
layer = 0
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.layers[layer].output)
intermediate_output = intermediate_layer_model.predict(input_train)
model.layers[layer]

In [7]:
embedding_matrix = intermediate_output
X = []
y = []
plot_X = []

for i in range(len(frame)):
    # prepare results for classification
    X.append(embedding_matrix[i,:].reshape(-1,1).squeeze().tolist())
    y.append(frame.most_frequent_label.values.tolist()[i])
    
    # prepare results for t-sne plot
    plot_X.append(embedding_matrix[i,:])

In [8]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2,verbose=1).fit_transform(plot_X)
ddf = pd.DataFrame(columns=['x','y','class','sentence', 'path'])
for i,x in enumerate(X_embedded):
    ddf.loc[i] = [x[0],x[1],frame.most_frequent_label.values.tolist()[i],frame.sentence.values.tolist()[i],frame.path.values.tolist()[i]]


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 3641 samples in 0.009s...
[t-SNE] Computed neighbors for 3641 samples in 1.196s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3641
[t-SNE] Computed conditional probabilities for sample 2000 / 3641
[t-SNE] Computed conditional probabilities for sample 3000 / 3641
[t-SNE] Computed conditional probabilities for sample 3641 / 3641
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 68.985962
[t-SNE] Error after 1000 iterations: 0.607244


### Plot

In [9]:
colormap = {'WORSE': 'red', 'BETTER': 'green', 'NONE': 'blue'}
colors = [colormap[x] for x in ddf['class']]

# GROUP tooltips
hover = HoverTool(tooltips=[
    ("Sentence", "@sentence"),
    ("Path", "@path"),
    ("label", "@label"),
])

p = figure(plot_width=900,  plot_height=900, tools="pan,wheel_zoom,box_zoom,reset,previewsave")
p.add_tools(hover)

p.cross(x='x', y='y', source=build_source('NONE', ddf),size=5,color='gray')
p.circle(x='x', y='y', source=build_source('WORSE', ddf) ,size=5,color='red')
p.circle(x='x', y='y', source=build_source('BETTER', ddf),size=5,color='green')


show(p)

### Classification Test

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
kf = StratifiedKFold(n_splits=5, random_state=1337)
for train_index, test_index in kf.split(X,y):
    X_train, X_test = np.array(X)[train_index], np.array(X)[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]

    log = LogisticRegression()
    log.fit(X_train, y_train)
    pred = log.predict(X_test)
    print(classification_report(pred,y_test,labels=['BETTER', 'WORSE', 'NONE']))